## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    toDate = dt.datetime(2019,4,18,15,15)
    fromDate = toDate - dt.timedelta(days = 1, hours=4)
    mode = "Preanalysis"
    papertrade = True


##  <span style="color:green">Trading Strategy </span>

In [2]:
%run "KiteConnect_Strategies_v2.ipynb"  

## Batch Analysis - Multithreaded

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
threaded = True
from multiprocessing import Pool
def threadedBatchAnalysis(x):
    global fig
    #fig = ""
    temp_data = getData(x, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
    #logging.debug(temp_data)

    fig = createPlot(x)
    
    plot_set1(temp_data)
    
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    #print(os.getcwd())
    plotData(x, False)

if __name__ == '__main__':
    portfolio = pd.DataFrame(['TCS', 'RIIL', 'HDFC','BAJAJ-AUTO'])
    multithreading = False
    if multithreading:
        p = Pool(10)
        p.map_async(threadedBatchAnalysis, portfolio[0])
        #p.apply_async(threadedBatchAnalysis, ['SBIN'])
        #portfolio
    else:
        for index, row in portfolio.iterrows():
            threadedBatchAnalysis(row[0])

## Batch Downloader for stock data

In [4]:
stock_list = pd.read_csv('data/zerodha_margin.csv')

stock_list.drop(['Serial No.','Unnamed: 3','Serial No..1','Stocks not allowed for MIS','Unnamed: 6', 'Reason', 'Stocks which were allowed but temporarily banned', 'Serial No..2'], axis=1, inplace=True)



stock_list.dropna(inplace=True)

stock_list.columns = ['symbol', 'margin']

stock_list['symbol'] = stock_list['symbol'].str.replace(':EQ','')

stock_list['margin'] = stock_list['margin'].str.replace('X','')

stock_list = stock_list.sort_values(['margin','symbol'])

portfolio = stock_list['symbol']

In [5]:
import time
logger.info("Resume download")
toDate = dt.datetime.now() -dt.timedelta(days=2*365)
fromDate = toDate - dt.timedelta(days=2*365)
freq = 'minute'
counter = 0
portfolio = []
for symbol in portfolio:
    try:
        getData(symbol, fromDate, toDate, exchange="NSE", freq=freq, force=False).head()
    except:
        logger.debug('Could not download: '+symbol)
        time.sleep(2)
        
    gc.collect()